In [ ]:
pip install jamo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import math
import json
import random
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
from jamo import h2j, j2hcj
from tqdm import tqdm
from pprint import pprint

In [ ]:
#좌표 세팅

kb = 'ㅣㆍㅡㄱㄴㄷㅂㅅㅈㅇ'

coords = {
    0: (0, 0),
    1: (72, 0),
    2: (144, 0),
    3: (0, 33),
    4: (72, 33),
    5: (144, 33),
    6: (0, 66),
    7:(72,66),
    8:(144,66),
    9:(72,99)
}

print(len(kb))

10


In [ ]:
#키 사이의 거리 계산

def distance(first, second):
    return math.hypot(second[0] - first[0], second[1] - first[1])

# test
print(distance(coords[0], coords[0]))
print(distance(coords[0], coords[9]))

0.0
122.41323457861898


In [ ]:
keys_per_finger = [[0,1,2,3,4,5,6,7,8,9]]
KEY_WIDTH = 216

distances = {i: {} for i in range(10)}

for keys in keys_per_finger:
    for i in keys:
        for j in keys:
            distances[i][j] = distance(coords[i], coords[j]) / KEY_WIDTH

distances

{0: {0: 0.0,
  1: 0.3333333333333333,
  2: 0.6666666666666666,
  3: 0.1527777777777778,
  4: 0.3666771884012246,
  5: 0.6839484584580627,
  6: 0.3055555555555556,
  7: 0.45218946100276963,
  8: 0.7333543768024492,
  9: 0.5667279378639768},
 1: {0: 0.3333333333333333,
  1: 0.0,
  2: 0.3333333333333333,
  3: 0.3666771884012246,
  4: 0.1527777777777778,
  5: 0.3666771884012246,
  6: 0.45218946100276963,
  7: 0.3055555555555556,
  8: 0.45218946100276963,
  9: 0.4583333333333333},
 2: {0: 0.6666666666666666,
  1: 0.3333333333333333,
  2: 0.0,
  3: 0.6839484584580627,
  4: 0.3666771884012246,
  5: 0.1527777777777778,
  6: 0.7333543768024492,
  7: 0.45218946100276963,
  8: 0.3055555555555556,
  9: 0.5667279378639768},
 3: {0: 0.1527777777777778,
  1: 0.3666771884012246,
  2: 0.6839484584580627,
  3: 0.0,
  4: 0.3333333333333333,
  5: 0.6666666666666666,
  6: 0.1527777777777778,
  7: 0.3666771884012246,
  8: 0.6839484584580627,
  9: 0.45218946100276963},
 4: {0: 0.3666771884012246,
  1: 0.1527

In [ ]:
# 손가락 초기 위치 설정

home_key_pos = [4]

home_keys = {}
for i, keys in enumerate(keys_per_finger):
    for key in keys:
        home_keys[key] = home_key_pos[i]

home_keys

{0: 4, 1: 4, 2: 4, 3: 4, 4: 4, 5: 4, 6: 4, 7: 4, 8: 4, 9: 4}

In [ ]:
kb = 'ㅣㆍㅡㄱㄴㄷㅂㅅㅈㅇ'

def genome_to_keyboard(genome):
    keyboard = {}
    for i, char in enumerate(genome):
        keyboard[char] = i
        
        if char == 'ㅂ':
            keyboard['ㅃ'] = i
        elif char == 'ㅈ':
            keyboard['ㅉ'] = i
        elif char == 'ㄷ':
            keyboard['ㄸ'] = i
        elif char == 'ㄱ':
            keyboard['ㄲ'] = i
        elif char == 'ㅅ':
            keyboard['ㅆ'] = i
        elif char == 'ㄱ':
            keyboard['ㅋ'] = i
        elif char == 'ㄴ':
            keyboard['ㄹ'] = i
        elif char == 'ㄷ':
            keyboard['ㅌ'] = i
        elif char == 'ㅂ':
            keyboard['ㅍ'] = i 
        elif char == 'ㅅ':
            keyboard['ㅎ'] = i 
        elif char == 'ㅈ':
            keyboard['ㅊ'] = i 
        elif char == 'ㅇ':
            keyboard['ㅁ'] = i 
    return keyboard

genome_to_keyboard(kb)

{'ㅣ': 0,
 'ㆍ': 1,
 'ㅡ': 2,
 'ㄱ': 3,
 'ㄲ': 3,
 'ㄴ': 4,
 'ㄹ': 4,
 'ㄷ': 5,
 'ㄸ': 5,
 'ㅂ': 6,
 'ㅃ': 6,
 'ㅅ': 7,
 'ㅆ': 7,
 'ㅈ': 8,
 'ㅉ': 8,
 'ㅇ': 9,
 'ㅁ': 9}

In [ ]:
def split_double_jamo(word):
    jamo = j2hcj(h2j(word))
    dic = {
        'ㅏ': 'ㅣㆍ','ㅓ': 'ㆍㅣ','ㅗ': 'ㆍㅡ','ㅜ': 'ㅡㆍ',
        'ㅕ': 'ㆍㆍㅣ','ㅑ': 'ㅣㆍㆍ','ㅛ': 'ㆍㆍㅡ',"ㅐ":'ㅣㆍㅣ',"ㅔ":'ㆍㅣㅣ',"ㅖ":'ㆍㆍㅣㅣ',
        "ㅒ":'ㅣㆍㆍㅣ',
        'ㅘ': 'ㆍㅡㅣㆍ', 'ㅙ': 'ㆍㅡㅣㆍㅣ', 'ㅚ': 'ㆍㅡㅣ', 'ㅝ': 'ㅡㆍㆍㅣ',
        'ㅞ': 'ㅡㆍㆍㅣㅣ', 'ㅟ': 'ㅡㆍㅣ', 'ㅢ': 'ㅡㅣ',
        'ㄳ': 'ㄱㅅ', 'ㄵ': 'ㄴㅈ', 'ㄶ': 'ㄴㅎ', 'ㄺ': 'ㄹㄱ',
        'ㄻ': 'ㄹㅁ', 'ㄼ': 'ㄹㅂ', 'ㄽ': 'ㄹㅅ', 'ㄾ': 'ㄹㅌ', 
        'ㄿ': 'ㄹㅍ', 'ㅀ': 'ㄹㅎ', 'ㅄ': 'ㅂㅅ'}

    for i in jamo:
        if i in dic.keys():
            jamo = jamo.replace(i,dic[i])
    return jamo

split_double_jamo('값싼 원두')

'ㄱㅣㆍㅂㅅㅆㅣㆍㄴ ㅇㅡㆍㆍㅣㄴㄷㅡㆍ'

In [ ]:
JSON_FILE_PATH = '/content/개인및관계.json'
DATA_LIMIT = 10

full_text = ''
legal_chars = 'ㅣㆍㅡㄱㄲㄴㄹㄷㄸㅂㅃㅅㅆㅈㅉㅇㅁ'

count = 0

with open(JSON_FILE_PATH,'rt',encoding='UTF8') as f:
    data = json.load(f)
    
    for d in tqdm(data['data']):
        if len(full_text) > DATA_LIMIT:
            break

        for body in d['body']:
            if body['utterance'].startswith('#@'): # 사진, 동영상 등 제거
                continue

            text = body['utterance'].replace('\n', ' ').strip()
            
            jamo_text = split_double_jamo(text)
            
            if any(char not in legal_chars for char in jamo_text): # 숫자, 영어 제거
                continue

            full_text += ' ' + jamo_text
            count += 1

print(count)
print(len(full_text))

NameError: ignored